<a href="https://colab.research.google.com/github/smerinobluewin/hello-world/blob/master/brown_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import random
import numpy as np
mu=0.05
sigma=2*mu
S1=1000000
C=50000
N=1*252
dt=1/252
for number in range(1,N+1):
  eps=random.normalvariate(mu=0,sigma=1)
  S=S1+S1*(mu*dt+sigma*eps*np.sqrt(dt)) - C*dt
  S1=S
print(S1)


1098661.768257481
